In [1]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, Rescaling, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data_dir = 'FF++'
output_dir = 'FF++_split_frames'
img_size = (224,224)
batch_size = 32

# Load Datasets

In [3]:
train_raw = keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, 'train'),
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary')

Found 1400 files belonging to 2 classes.


In [4]:
val_raw = keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, 'val'),
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary')

Found 300 files belonging to 2 classes.


In [5]:
test_raw = keras.utils.image_dataset_from_directory(
    os.path.join(output_dir, 'test'),
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary', 
    shuffle=False)

Found 300 files belonging to 2 classes.


In [6]:
# autotune automatically decides how many batches to pre-fetch based on cpu, gpu, memory etc.
train = train_raw.prefetch(tf.data.AUTOTUNE)
val = val_raw.prefetch(tf.data.AUTOTUNE)
test = test_raw.prefetch(tf.data.AUTOTUNE)

# Simple CNN Model

In [7]:
model = Sequential()
model.add(Input(shape=img_size+(3,)))
model.add(Rescaling(1./255))
model.add(Conv2D(32, 3, strides=1, padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Conv2D(64, 3, strides=1, padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Conv2D(128, 3, strides=1, padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=5,
                      restore_best_weights=True,
                      verbose=1)

In [10]:
model.fit(train, validation_data=val,
          epochs=500, callbacks=[estop],
          verbose=1)

Epoch 1/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 29s 571ms/step - accuracy: 0.5336 - loss: 0.7608 - val_accuracy: 0.5067 - val_loss: 0.7097
Epoch 2/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 552ms/step - accuracy: 0.6086 - loss: 0.6481 - val_accuracy: 0.4800 - val_loss: 0.7158
Epoch 3/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 24s 536ms/step - accuracy: 0.6721 - loss: 0.5980 - val_accuracy: 0.5700 - val_loss: 0.7664
Epoch 4/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 563ms/step - accuracy: 0.7279 - loss: 0.5392 - val_accuracy: 0.5500 - val_loss: 0.8767
Epoch 5/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 558ms/step - accuracy: 0.7579 - loss: 0.4934 - val_accuracy: 0.5767 - val_loss: 0.9517
Epoch 6/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 568ms/step - accuracy: 0.7821 - loss: 0.4392 - val_accuracy: 0.5600 - val_loss: 1.0390
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [11]:
test_loss, test_accuracy = model.evaluate(test, verbose=0)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

Test loss: 0.7349957227706909
Test accuracy: 0.47999998927116394


**Simple CNN model gave a (frame-level) test accuracy of 48%.**

In [12]:
model.save('artifacts/baseline_cnn_model.keras')

In [13]:
predictions = model.predict(test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step


In [14]:
file_names = test_raw.file_paths
class_names = test_raw.class_names

In [18]:
file_names

['FF++_split_frames\\test\\fake\\01_11__meeting_serious__9OM3VE0Y_0.jpg',
 'FF++_split_frames\\test\\fake\\01_11__meeting_serious__9OM3VE0Y_1.jpg',
 'FF++_split_frames\\test\\fake\\01_11__meeting_serious__9OM3VE0Y_2.jpg',
 'FF++_split_frames\\test\\fake\\01_11__meeting_serious__9OM3VE0Y_3.jpg',
 'FF++_split_frames\\test\\fake\\01_11__meeting_serious__9OM3VE0Y_4.jpg',
 'FF++_split_frames\\test\\fake\\01_12__outside_talking_pan_laughing__TNI7KUZ6_0.jpg',
 'FF++_split_frames\\test\\fake\\01_12__outside_talking_pan_laughing__TNI7KUZ6_1.jpg',
 'FF++_split_frames\\test\\fake\\01_12__outside_talking_pan_laughing__TNI7KUZ6_2.jpg',
 'FF++_split_frames\\test\\fake\\01_12__outside_talking_pan_laughing__TNI7KUZ6_3.jpg',
 'FF++_split_frames\\test\\fake\\01_12__outside_talking_pan_laughing__TNI7KUZ6_4.jpg',
 'FF++_split_frames\\test\\fake\\02_03__walking_outside_cafe_disgusted__QH3Y0IG0_0.jpg',
 'FF++_split_frames\\test\\fake\\02_03__walking_outside_cafe_disgusted__QH3Y0IG0_1.jpg',
 'FF++_split_fram

In [19]:
class_names

['fake', 'real']

In [15]:
y_true = np.concatenate([np.array(y) for x,y in test_raw], axis=0)

In [16]:
y_true.shape

(300, 1)

In [17]:
y_pred = np.array([[1] if pred[0]>=0.5 else [0] for pred in predictions])
y_pred.shape

(300, 1)

In [18]:
frame_pred_df = pd.DataFrame(data={
    'frame':file_names,
    'actual_class':[class_names[int(cls[0])] for cls in y_true],
    'predicted_class':[class_names[int(cls[0])] for cls in y_pred]
})

In [19]:
frame_pred_df.sample(20)

,frame,actual_class,predicted_class
47,FF++_split_frames\test\fake\03_09__secret_conv...,fake,fake
77,FF++_split_frames\test\fake\05_16__walk_down_h...,fake,fake
242,FF++_split_frames\test\real\11__exit_phone_roo...,real,fake
174,FF++_split_frames\test\real\02__walking_down_s...,real,fake
293,FF++_split_frames\test\real\14__secret_convers...,real,fake
111,FF++_split_frames\test\fake\07_02__talking_ang...,fake,fake
190,FF++_split_frames\test\real\05__walking_down_s...,real,fake
204,FF++_split_frames\test\real\06__outside_talkin...,real,fake
291,FF++_split_frames\test\real\14__secret_convers...,real,fake
39,FF++_split_frames\test\fake\03_07__walk_down_h...,fake,fake


In [ ]:
# for file in file_names:
#     print(os.path.splitext(os.path.basename(file)))

In [20]:
video_pred_df = {'video':[], 'actual_class':[], 'predicted_class':[]}
for i, file in enumerate(file_names):
    dirname = os.path.dirname(file)
    f = os.path.splitext(os.path.basename(file))[0][:-2]
    if f not in video_pred_df['video']:
        video_pred_df['video'].append(f)
        video_pred_df['actual_class'].append(class_names[int(y_true[i][0])])
        idxs = [file_names.index(os.path.join(dirname,f+f"_{c}.jpg")) for c in range(5)]
        majority_pred = 1 if sum(int(y_pred[idx][0]) for idx in idxs)>2.5 else 0
        video_pred_df['predicted_class'].append(class_names[majority_pred])

In [21]:
video_pred_df = pd.DataFrame(video_pred_df)
video_pred_df

,video,actual_class,predicted_class
0,01_11__meeting_serious__9OM3VE0Y,fake,fake
1,01_12__outside_talking_pan_laughing__TNI7KUZ6,fake,fake
2,02_03__walking_outside_cafe_disgusted__QH3Y0IG0,fake,fake
3,02_07__meeting_serious__1JCLEEBQ,fake,fake
4,02_09__kitchen_pan__HIH8YA82,fake,fake
5,02_15__walking_and_outside_surprised__I8G2LWD1,fake,fake
6,03_06__podium_speech_happy__83ABVHC3,fake,fake
7,03_07__walk_down_hall_angry__IFSURI9X,fake,fake
8,03_09__outside_talking_still_laughing__RCETIXYL,fake,fake
9,03_09__secret_conversation__RCETIXYL,fake,fake


In [22]:
accuracy = len(video_pred_df[video_pred_df['actual_class']==video_pred_df['predicted_class']])/len(video_pred_df)
print(accuracy)

0.4666666666666667


**With majority voting frame predictions are combined to obtain video predictions. The video-level test accuracy achieved is 46.7%.**